# Define and assign bondwire profiles

In [ ]:
%run helpers/example_setup.ipynb

## Basic setup of empty EDB

In [ ]:
"""Create Database, Cell, Layout, Net"""

# cells
cell_start = Cell.create(db, CellType.CIRCUIT_CELL, "Start")
cell_start.is_blackbox = True
cell_end = Cell.create(db, CellType.CIRCUIT_CELL, "End")
cell_end.is_blackbox = True

# Layouts
layout_start = cell_start.layout
layout_end = cell_end.layout

# Net
net = Net.create(layout, "signal")
net_start = Net.create(layout_start, "start_signal")
net_end = Net.create(layout_end, "end_signal")

## Setup Layers for the different cells

In [ ]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layout's layer collection
lc = layout.layer_collection
layout.layer_collection.add_layer_top(
    StackupLayer.create("signal", LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
)

# Layers for additional cells
lcs = layout_start.layer_collection
lcs.add_layer_top(
    StackupLayer.create("start_signal", LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
)
lce = layout_end.layer_collection
lce.add_layer_top(
    StackupLayer.create("end_signal", LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
)

# Retrieve layers collection and layers
signal_lyr = lc.find_by_name("signal")
start_signal_lyr = lcs.find_by_name("start_signal")
end_signal_lyr = lce.find_by_name("end_signal")

# Print layers created for lc
print("LC", [l.name for l in layout.layer_collection.get_layers()])
# Print layers created for lcs
print("LCS", [l.name for l in layout_start.layer_collection.get_layers()])
# Print layers created for lce
print("LCE", [l.name for l in layout_end.layer_collection.get_layers()])

## Create geometry

In [ ]:
"""Create Primitive objects"""

# Rectangles
def mm(val):
    return val * 1e-3

Rectangle.create(
    layout=layout_start,
    layer=start_signal_lyr,
    net=net_start,
    rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
    param1=mm(0.0),
    param2=mm(0.0),
    param3=mm(0.30),
    param4=mm(0.30),
    corner_rad=0.0,
    rotation=0.0,
)

Rectangle.create(
    layout=layout_end,
    layer=end_signal_lyr,
    net=net_end,
    rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
    param1=mm(0.0),
    param2=mm(0.0),
    param3=mm(0.20),
    param4=mm(0.20),
    corner_rad=0.0,
    rotation=0.0,
)

# Retrieve the primitives
primitives_created = layout_start.primitives + layout_end.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created])

## Create the cell instances

In [ ]:
cis = CellInstance.create(layout, 'CellInstanceStart', layout_start)
cie = CellInstance.create(layout, 'CellInstanceEnd', layout_end)
cie.location = mm(0.60), mm(0.05)

## Create the bondwire definition

In [ ]:
# Create an APD Bondwire Definition
wire_0 = ApdBondwireDef.create(db, "Wire_0")

# Bondwire Parameters name (nm) diameter(dia) and material(mat) should be set as strings(quoted)
parameters_str = "bwd(nm='Wire_0', ven=true, for=true, dia='2.54e-05', mat='gold',\
col=0, vis=true, dih=0, nfc=6, seg(ht=0, hv=0, vt=0, vv=0.0001))"

# Set parameters
wire_0.set_parameters(parameters_str)

## Create the bondwire material

In [ ]:
gold_mat = MaterialDef.create(db,"gold")
gold_mat.set_property(MaterialProperty.CONDUCTIVITY, 41000000)
gold_mat.set_property(MaterialProperty.PERMITTIVITY, 1)
gold_mat.set_property(MaterialProperty.PERMEABILITY, 0.99996)
gold_mat.set_property(MaterialProperty.DIELECTRIC_LOSS_TANGENT, 0)

## Create the bondwire

In [ ]:
# Bondwire
bondwire = Bondwire.create(
    layout=layout,
    bondwire_type=BondwireType.APD,
    definition_name="Wire_0",
    placement_layer=signal_lyr.name,
    width=mm(0.01),
    material="gold",
    start_context=cis,
    start_layer_name=start_signal_lyr.name,
    start_x=mm(0.15),
    start_y=mm(0.15),
    end_context=cie,
    end_layer_name=end_signal_lyr.name,
    end_x=mm(0.55),
    end_y=mm(0.15),
    net=net,
)
bondwire.cross_section_type = BondwireCrossSectionType.RECTANGLE
bondwire.cross_section_height = mm(0.01)
bondwire.set_traj(mm(0.15), mm(0.15), mm(0.70), mm(0.15))

In [ ]:
%run helpers/example_teardown.ipynb